In [16]:
import numpy as np
import scipy.linalg as linalg

from pulp import *

# (1) Linear Optimization Problem (LP)

## Fundamental Theorem

### (A) Optimal Solution

In [2]:
_ = LpProblem(name='optimal', sense=LpMaximize)

x1 = LpVariable('x1', lowBound=0.0)
x2 = LpVariable('x2', lowBound=0.0)

_ += (2 * x1) + (3 * x2)
_ += x1 + (3 * x2) <= 9, 'const. 1'
_ += x1 + x2 <= 4, 'const. 2'
_ += x1 + x2 <= 10, 'const. 3'

In [3]:
_

optimal:
MAXIMIZE
2*x1 + 3*x2 + 0
SUBJECT TO
const._1: x1 + 3 x2 <= 9

const._2: x1 + x2 <= 4

const._3: x1 + x2 <= 10

VARIABLES
x1 Continuous
x2 Continuous



In [4]:
_.solve()

1

In [5]:
LpStatus[_.status], value(_.objective)

('Optimal', 10.5)

In [6]:
for x in _.variables():
    print(f'{x.name} = {value(x)}')

x1 = 1.5
x2 = 2.5


### (B) Infeasible Problem

In [7]:
_ = LpProblem(name='infeasible', sense=LpMaximize)

x1 = LpVariable('x1', lowBound=0.0)
x2 = LpVariable('x2', lowBound=0.0)

_ += x1 + x2
_ += x1 - x2 <= -1, 'const. 1'
_ += (-1 * x1) + x2 <= -1, 'const. 2'

In [8]:
_.solve()

-1

In [9]:
LpStatus[_.status], value(_.objective)

('Infeasible', 62500000000000.0)

In [10]:
for x in _.variables():
    print(f'{x.name} = {value(x)}')

x1 = 31250000000000.0
x2 = 31250000000000.0


### (C) Unbounded Problem

In [11]:
_ = LpProblem(name='unbounded', sense=LpMaximize)

x1 = LpVariable('x1', lowBound=0.0)
x2 = LpVariable('x2', lowBound=0.0)

_ += x1 + x2
_ += (-2 * x1) + x2 <= 2, 'const. 1'
_ += x1 - (2 * x2) <= -2, 'const. 2'

In [12]:
_.solve()

-2

In [13]:
LpStatus[_.status], value(_.objective)

('Unbounded', 0.0)

In [14]:
for x in _.variables():
    print(f'{x.name} = {value(x)}')

x1 = 0.0
x2 = 0.0


## Algorithms

### (A) Simplex Method

In [17]:
A = np.array([[2,2,-1],[2,-2,3],[0,2,-1]])
c = np.array([4,3,5])
b = np.array([6,8,4])

In [20]:
import numpy as np
import scipy.linalg as linalg
MEPS = 1.0e-10

def lp_RevisedSimplex(c,A,b):
    np.seterr(divide='ignore')
    (m,n) = A.shape # m は A の行数，n は A の列数
    AI = np.hstack((A,np.identity(m)))
    c0 = np.r_[c,np.zeros(m)]
    basis = [n+i for i in range(m)]
    nonbasis = [j for j in range(n)]

    while True:
        y = linalg.solve(AI[:,basis].T, c0[basis])
        cc = c0[nonbasis]-np.dot(y,AI[:,nonbasis])

        if np.all(cc <= MEPS): # 最適性判定
            x = np.zeros(n+m)
            x[basis] = linalg.solve(AI[:,basis], b)
            print('Optimal')
            print('Optimal value =',np.dot(c0[basis],x[basis]))
            for i in range(m):
                print('x',i, '=', x[i])
            break
        else:
            s = np.argmax(cc)
        d = linalg.solve(AI[:,basis], AI[:,nonbasis[s]])
        if np.all(d <= MEPS): # 非有界性判定
            print('Unbounded')
            break
        else:
            bb = linalg.solve(AI[:,basis], b)
            ratio = bb/d
            ratio[ratio<-MEPS] = np.inf
            r = np.argmin(ratio)
            # 基底と非基底の入れ替え
            nonbasis[s], basis[r] = basis[r], nonbasis[s]

In [21]:
lp_RevisedSimplex(c,A,b)

Optimal
Optimal value = 45.0
x 0 = 0.0
x 1 = 4.999999999999999
x 2 = 6.0


### (B) Interior Point Method